In [101]:
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

###############################################
# Import Miscellaneous Assets
###############################################
import numpy as np

# Data manipulation
import pandas as pd
import numpy as np

# Modeling
import lightgbm as lgb

# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.plotting import scatter_matrix
%matplotlib inline


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from dateutil.parser import parse
from datetime import datetime
from scipy.stats import norm

# import all what you need for machine learning
import sklearn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import warnings
from datetime import datetime
from functools import partial
from pprint import pprint as pp
from tqdm import tqdm, tqdm_notebook

import tensorflow as tf
import math
from hyperopt import hp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

import xgboost as xgb
from sklearn.metrics import accuracy_score

from numpy import loadtxt
from xgboost import XGBClassifier

import lightgbm as lgb
from hyperopt import STATUS_OK

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers

from sklearn.metrics import accuracy_score
import os
from sklearn import preprocessing


###### ESSIENTIAL CODE ###########
train = pd.read_csv("/kaggle/input/datamaestro2020/astro_train.csv")
test = pd.read_csv("/kaggle/input/datamaestro2020/astro_test.csv")
sample = pd.read_csv("/kaggle/input/datamaestro2020/sample_submission.csv")

### useless thing ifigured all out in different notebook, data_maestro_1 dekho ab
del train["id"]
del test["id"]

del train["rerun"]
del test["rerun"]
del train["skyVersion"]
del test["skyVersion"]
del train["run"]
del test["run"]
del train["camCol"]
del test["camCol"]

### minmax scaling karna hai but it get converted to different data type 
### bhavika tera manually kiya hua function bhej lol

train.to_csv('train_new.csv', header=True, index=False)
test.to_csv('test_new.csv', header=True, index=False)



In [102]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from bayes_opt import BayesianOptimization
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,31 mins 52 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.4
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_unknownUser_1mu23a
H2O cluster total nodes:,1
H2O cluster free memory:,2.827 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [103]:
data = h2o.upload_file("/kaggle/working/train_new.csv")
train_cols = [x for x in data.col_names if x not in ['class']]
target = "class"
train, test = data.split_frame(ratios=[0.7])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [104]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = H2OGradientBoostingEstimator(nfolds=5,**params)
    model.train(x=train_cols, y=target, training_frame=train)
    return -model.rmse()

In [105]:
bounds = {
    'max_depth':(3,8),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [106]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=1)
# n_iter = 50 #basic

In [107]:
optimizer.max

{}

In [108]:
#set manually to optimiser.max idk how to do directly cheap trick lol
#maxdepth and ntrees int karaychi approx

#defining
model = H2OGradientBoostingEstimator(nfolds=5, 
                                     col_sample_rate= 0.5558780634133013,
                                     learn_rate = 0.00411004654338743,
                                     max_depth = 7,
                                     min_rows  = 20.776334680067137,
                                     ntrees = 268,
                                     sample_rate = 0.7055658501190278
                                    )

#traneeeee
model.train(x=train_cols, y=target, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [100]:
#test = pd.read_csv("/kaggle/input/datamaestro2020/astro_test.csv")

test = h2o.upload_file("/kaggle/working/test_new.csv")

#test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [97]:
#a lotta random convesions here which i tuk too long to figar out

predictions = model.predict(test)
pdr = predictions.as_data_frame().to_numpy()
p = np.round(pdr).astype(int)

#allah ho gaya finally

submission_df = pd.DataFrame(columns=['id', 'class'])
submission_df['id'] = sample['id']
submission_df['class'] = p
submission_df.to_csv('bayesian.csv', header=True, index=False)

#dekhlo ab kyahi, test noi banaya banana hai toh use own brain
submission_df.head(10)

,id,class
0,45000,1
1,45001,1
2,45002,1
3,45003,1
4,45004,1
...,...,...
95,45095,1
96,45096,0
97,45097,1
98,45098,1
